## Exercice séance 4

Tous les exercices sont sur les données du fichier `CSS_openalex.csv` diponible ici : https://filesender.renater.fr/?s=download&token=01def21d-738d-48f1-860a-820c37fab6d7


Ces données représentent toutes les publications scientifiques de la base openalex qui mentionnent `computational social science` dans leur titre ou abstract.

## Exercice 1

- Quelle est la distribution des types de publications (`type`) du dataset en pourcentages
- Renommer la modalité `Other` en `Autre` (regarder du côté de .replace)
- Renomer la variable `primary_location.source.display_name` en `source` (regarder du côté de .rename)

In [ ]:
# Commencer par importer pandas
import pandas as pd

# Et lire les données
df = pd.read_csv("../../data/CSS_openalex.csv", low_memory=False)
# low_memory=False n'est pas obligatoire mais me permet d'éviter le message d'erreur

> Internally process the file in chunks, resulting in lower memory use while parsing, but possibly mixed type inference. To ensure no mixed types either set False, or specify the type with the dtype parameter. Note that the entire file is read into a single DataFrame regardless, use the chunksize or iterator parameter to return the data in chunks. (Only valid with C parser).

### 1.1 Distribution

In [ ]:
# Je veux donc le nombre de publication par "type"
df["type"].value_counts()

In [ ]:
# En fait je veux les fréquences :
# donc normaliser les valeurs : normalize=True
df["type"].value_counts(normalize=True)

In [ ]:
# Je les veux même en pourcentage ces fréquences : *100
df["type"].value_counts(normalize=True) * 100

In [ ]:
# Et je peux arrondir :
# la dernière valeur me permet de préciser le nb de chiffre après la virgule
round(100 * df["type"].value_counts(normalize=True), 2)

### 1.2 Renommer la modalité `Other` en `Autre` (regarder du côté de .replace)


In [ ]:
# Recoder en utilisant replace

df["type"].replace({"other": "autre"}, inplace=True)
# je peux utiliser inplace=True pour modifier directement le dataframe

# l'alternative serait ça (que l'on retrouve pas mal) :
# df["type"] = df["type"].replace({"other" : "autre"})

# À DISCUTER : inplace est même désormais déconseillé ?
# https://jorisvandenbossche.github.io/pandas-website-preview///pdeps/0008-inplace-methods-in-pandas.html

In [ ]:
df["type"].value_counts()

In [ ]:
# Si on n'a qu'une modification, on peut se contenter de passer nos valeurs dans la parenthèse
df["type"] = df["type"].replace("preprint", "pré-publication")
df["type"].value_counts()

In [ ]:
# mais le dictionnaire est pratique dès que l'on veut modifier plusieurs éléments :
df["type"] = df["type"].replace(
    {
        "article": "article",
        "preprint": "pré-publication",
        "book-chapter": "chapitre de livre",
        "book": "livre",
        "review": "revue de littérature",
    }
)
df["type"].value_counts()

### 1.3 - Renomer la variable `primary_location.source.display_name` en `source` (regarder du côté de .rename)

In [ ]:
df = df.rename(columns={"primary_location.source.display_name": "source"})

In [ ]:
df["source"]

## Exercice 2

- Ajouter une colonne qui correspond au nombre de mots des titres (`title`) de chaque référence.
- Calculer le nombre de mots moyens et la variance par `source`.
- Quelle est la `source` qui a le plus de mots, le moins de mots ?


### 2.1 Ajouter une colonne qui correspond au nombre de mots des titres (`title`) de chaque référence.


In [ ]:
# Facile à faire directement en pandas :
df["title_nb_mots"] = df["title"].str.split().str.len()
df["title_nb_mots"]

In [ ]:
# Attention, s'il n'y a rien il ne met pas 0 mais renvoie un NaN
df["title_nb_mots"].isna().sum()

# j'aurai pu lui demander explicitement de renvoyer 0 en ajoutant .fillna(0)

In [ ]:
# On savait déjà le faire en faisant une fonction
def compter_mots(texte):
    texte = str(texte)
    mots = texte.split()
    nb_mots = len(mots)
    return nb_mots

In [ ]:
# Que l'on peut appliquer avec apply :
df["title"].apply(compter_mots)

In [ ]:
# pareil que ça avec de la compréhension de liste :
df["title"].apply(lambda x: len(str(x).split()))

### 2.2 Calculer le nombre de mots moyens et la variance par `source`.

In [ ]:
# je veux donc grouper par source
# puis calculer la moyenne et la variance du nombre de mots
df.groupby("source")["title_nb_mots"].mean()

In [ ]:
df.groupby("source")["title_nb_mots"].var()

In [ ]:
# Pour mettre tout en même temps avec .agg()
df.groupby("source")["title_nb_mots"].agg(["mean", "median", "var", "std"])

### 2.3 Quelle est la `source` qui a le plus de mots, le moins de mots ?

In [ ]:
df[df["title_nb_mots"] == df["title_nb_mots"].max()]

In [ ]:
df[df["title_nb_mots"] == df["title_nb_mots"].min()]

In [ ]:
df.loc[df["title_nb_mots"].idxmax()]

In [ ]:
df.loc[df["title_nb_mots"].idxmax(), "source"]

`If multiple values equal the maximum, the first row label with that value is returned.`

In [ ]:
df.loc[df["title_nb_mots"].idxmin()]

In [ ]:
df["title_nb_mots"].idxmin()

## Exercice 4

- Découper le montant des frais de publication en USD (`apc_list.value_usd`) en quartiles avec la méthode `qcut`
- Quelle est la référence qui a payé le montant le plus élevé ?

### 4.1 - Découper le montant des frais de publication en USD (`apc_list.value_usd`) en quartiles avec la méthode `qcut`

In [ ]:
df["apc_quartile"] = pd.qcut(
    df["apc_list.value_usd"], [0, 0.25, 0.5, 0.75, 1], ["Q1", "Q2", "Q3", "Q4"]
)

In [ ]:
df["apc_quartile"].value_counts(normalize=True).sort_index() * 100

In [ ]:
df["apc_quartile"].value_counts(normalize=True, dropna=False).sort_index() * 100

### 4.2 Quelle est la référence qui a payé le montant le plus élevé ?

In [ ]:
df.loc[[df["apc_list.value_usd"].idxmax()], ["title", "source", "apc_list.value_usd"]]

`If multiple values equal the maximum, the first row label with that value is returned.`

In [ ]:
# je peux donc selectionner dans mon dataframe
# l'ensemble des références qui remplissent la condition
df[df["apc_list.value_usd"] == df["apc_list.value_usd"].max()]

In [ ]:
# et avec un peu de barbarie et de crochets renvoyer ce qui m'intéresse:
df[df["apc_list.value_usd"] == df["apc_list.value_usd"].max()][
    ["title", "source", "apc_list.value_usd"]
]

## Exercice 5

- Compter le nombre de publications par auteurs (`authorships.author.display_name`)
- Conseil :
    - spliter les auteurs sur "|"
    - regarder ensuite du côté de `.explode()`et de `.value_counts()`
- Faire un barplot des 5 principaux publiants


### 5.1 Compter le nombre de publications par auteurs (`authorships.author.display_name`)

Plus prise de tête ici, demande à aller plus loin et manipuler un peu tout ça.

In [ ]:
# En fait nos auteurs sont plusieurs par cellule :
df["authorships.author.display_name"]

In [ ]:
# Avant de les compter je dois donc les séparer
# Je peux utiliser str.split(), équivalent pandas de split()
df["auteurs"] = df["authorships.author.display_name"].str.split("|")
df["auteurs"]

In [ ]:
# puis exploser ça pour avoir un auteur par ligne
# On a donc un colonne bien plus longue
all_authors = df["auteurs"].explode()
all_authors

In [ ]:
# Et enfin on peut faire un value_counts() dessus:
all_authors.value_counts()


### 5.2 Faire un barplot des 5 principaux publiants


In [ ]:
all_authors.value_counts()[0:5].plot(kind="bar")